In [4]:
import qiskit

from PatchedMeasCal.edge_bfs import CouplingMapGraph
from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.fake_backends import Square, Hexagonal16, FullyConnected

import qiskit.tools.jupyter